In [1]:
# Chargement des Données dans Elasticsearch depuis un Notebook

# Dans ce notebook, nous allons explorer comment charger un jeu de données dans Elasticsearch à partir d'un fichier dat.
# Nous avons une installation cloud Elasticsearch, ce qui signifie que nous devrons utiliser les informations d'identification fournies 
# par l'utilisateur elastic pour nous connecter, et le mot de passe sera celui de l'API Elasticsearch.


In [2]:
# Étape 1 : Configuration de l'Environnement

# Tout d'abord, nous devons configurer notre environnement en important les bibliothèques nécessaires 
# et en définissant nos informations d'authentification Elasticsearch.

In [4]:
# import os
from elasticsearch import Elasticsearch, helpers

# Récupérer le mot de passe à partir d'une variable d'environnement
# password = os.environ.get("ENV_PASSWORD")
#le mot de passe de l' API elastic
password = "Qn5HOUiNO7V3GWpgxXok1aeE"

# Définir l'URL Elasticsearch (IL S' AGIT DE L' URL  elasticsearch endpoint de notre deploiement movielens)
elastic_url = "https://2830fc520a954b858492459c95e36087.us-central1.gcp.cloud.es.io:443"

# Créer une connexion Elasticsearch
client = Elasticsearch(hosts=[elastic_url], basic_auth=('elastic', password))


In [4]:
# Étape 2 : Chargement des Données

# Maintenant que notre environnement est configuré, nous pouvons procéder au chargement des données. 
# Nous allons charger un jeu de données à partir des fichiers dat dans Elasticsearch.

In [8]:
# Fonction pour charger les données ratings du fichier ratings.dat dans Elasticsearch
def load_ratings_to_elasticsearch(filename, index_name):
    # Lecture du fichier ratings.dat
    with open(filename, 'r') as file:
        # Parcourir chaque ligne du fichier
        for line in file:
            # Séparer les champs en utilisant '::' comme séparateur
            fields = line.strip().split("::")
            # Créer un document Elasticsearch à partir des champs
            doc = {
                "user_id": int(fields[0]),
                "movie_id": int(fields[1]),
                "rating": float(fields[2]),
                "timestamp": int(fields[3])
            }
            # Indexer le document dans Elasticsearch
            yield {
                "_index": index_name,
                "_source": doc
            }

# Nom de l'index Elasticsearch
index_name = "ratings"

# Chemin vers le fichier ratings.dat
ratings_file = "/home/chelsie/ml-1m/ratings.dat"

# Charger les données dans Elasticsearch en utilisant la fonction d'aide bulk
helpers.bulk(client, load_ratings_to_elasticsearch(ratings_file, index_name))

print("Les données ratings ont été chargées dans Elasticsearch avec succès.")


Les données ratings ont été chargées dans Elasticsearch avec succès.


In [9]:
# Fonction pour charger les données users du fichier users.dat dans Elasticsearch
def load_users_to_elasticsearch(filename, index_name):
    # Lecture du fichier users.dat
    with open(filename, 'r') as file:
        # Parcourir chaque ligne du fichier
        for line in file:
            # Séparer les champs en utilisant '::' comme séparateur
            fields = line.strip().split("::")
            # Créer un document Elasticsearch à partir des champs
            doc = {
                "user_id": int(fields[0]),
                "gender": fields[1],
                "age": int(fields[2]),
                "occupation": fields[3],
                "zipcode": fields[4]
            }
            # Indexer le document dans Elasticsearch
            yield {
                "_index": index_name,
                "_source": doc
            }

# Nom de l'index Elasticsearch
index_name = "users"

# Chemin vers le fichier users.dat
users_file = "/home/chelsie/ml-1m/users.dat"

# Charger les données dans Elasticsearch en utilisant la fonction d'aide bulk
helpers.bulk(client, load_users_to_elasticsearch(users_file, index_name))

print("Les données users ont été chargées dans Elasticsearch avec succès.")


Les données users ont été chargées dans Elasticsearch avec succès.


In [6]:
# Fonction pour charger les données movies du fichier movies.dat dans Elasticsearch
def load_movies_to_elasticsearch(filename, index_name):
    # Lecture du fichier movies.dat
    with open(filename, 'r', encoding="latin1") as file:
        # Parcourir chaque ligne du fichier
        for line in file:
            # Séparer les champs en utilisant '::' comme séparateur
            fields = line.strip().split("::")
            # Créer un document Elasticsearch à partir des champs
            doc = {
                "movie_id": int(fields[0]),
                "title": fields[1],
                "genres": fields[2].split("|")
            }
            # Indexer le document dans Elasticsearch
            yield {
                "_index": index_name,
                "_source": doc
            }

# Nom de l'index Elasticsearch
index_name = "movies"

# Chemin vers le fichier movies.dat
movies_file = "/home/chelsie/ml-1m/movies.dat"

# Charger les données dans Elasticsearch en utilisant la fonction d'aide bulk
helpers.bulk(client, load_movies_to_elasticsearch(movies_file, index_name))

print("Les données movies ont été chargées dans Elasticsearch avec succès.")


Les données movies ont été chargées dans Elasticsearch avec succès.


In [18]:
# Étape 3 : Récupération des Données depuis Elasticsearch

# Maintenant que nous avons établi la connexion avec Elasticsearch et construit notre requête, 
# nous pouvons procéder à la récupération des données depuis Elasticsearch.

# Nous utilisons la méthode search de notre client Elasticsearch pour exécuter la requête et récupérer les résultats.
# Si le nombre de document à importer est supérieure à la limite par défaut de 10 000, 
# nous pouvons utiliser la méthode scann pour récupérer nos documents par lots de 10 000.

In [5]:
# Création d'une session Spark
spark = SparkSession.builder \
    .appName("Chargement des données de Elasticsearch vers PySpark DataFrame") \
    .getOrCreate()

In [6]:
# Requête Elasticsearch pour récupérer tous les documents de l'index "users"
query = {
    "query": {
        "match_all": {}
    },
    "size": 6040
}

# Récupération des données Elasticsearch
es_response = client.search(index="users", body=query)  

# Convertir les résultats en une liste de dictionnaires
documents = [hit["_source"] for hit in es_response["hits"]["hits"]]

# Convertir la liste de dictionnaires en DataFrame Spark
es_users_df = spark.createDataFrame(documents)

# Afficher les premières lignes du DataFrame
es_users_df.show()

print(es_users_df.count()) 



+---+------+----------+-------+-------+
|age|gender|occupation|user_id|zipcode|
+---+------+----------+-------+-------+
|  1|     F|        10|      1|  48067|
| 56|     M|        16|      2|  70072|
| 25|     M|        15|      3|  55117|
| 45|     M|         7|      4|  02460|
| 25|     M|        20|      5|  55455|
| 50|     F|         9|      6|  55117|
| 35|     M|         1|      7|  06810|
| 25|     M|        12|      8|  11413|
| 25|     M|        17|      9|  61614|
| 35|     F|         1|     10|  95370|
| 25|     F|         1|     11|  04093|
| 25|     M|        12|     12|  32793|
| 45|     M|         1|     13|  93304|
| 35|     M|         0|     14|  60126|
| 25|     M|         7|     15|  22903|
| 35|     F|         0|     16|  20670|
| 50|     M|         1|     17|  95350|
| 18|     F|         3|     18|  95825|
|  1|     M|        10|     19|  48073|
| 25|     M|        14|     20|  55113|
+---+------+----------+-------+-------+
only showing top 20 rows



6040


In [7]:
from elasticsearch.helpers import scan

# Requête Elasticsearch pour récupérer tous les documents de l'index "ratings"
query = {
    "query": {
        "match_all": {}
    }
}

# Utilisation de la fonction scan pour obtenir les résultats de la recherche par lots
results = scan(client, query=query, index="ratings", size=10000)

# Créer une liste pour stocker les données des documents
data = []

# Parcourir les résultats
for res in results:
    data.append(res['_source'])

# Créer un DataFrame Spark à partir de la liste de dictionnaires
es_ratings_df = spark.createDataFrame(data)

# Afficher les premières lignes du DataFrame
es_ratings_df.show()
print(es_ratings_df.count()) 


24/04/12 05:11:34 WARN TaskSetManager: Stage 4 contains a task of very large size (11175 KiB). The maximum recommended task size is 1000 KiB.


+--------+------+----------+-------+
|movie_id|rating| timestamp|user_id|
+--------+------+----------+-------+
|    2066|   3.0|1042049967|    146|
|     804|   1.0| 977348138|    146|
|    1193|   4.0| 979940868|    146|
|    1267|   4.0| 977349660|    146|
|    1269|   5.0| 977434285|    146|
|    1196|   4.0| 977336700|    146|
|    1197|   4.0| 977341318|    146|
|    1198|   5.0| 979939805|    146|
|    1199|   4.0| 979939730|    146|
|     592|   3.0| 979940007|    146|
|     594|   5.0| 977348818|    146|
|     595|   5.0| 977348788|    146|
|     596|   3.0| 977348869|    146|
|     597|   4.0|1022004006|    146|
|    2212|   4.0|1010690287|    146|
|    2140|   2.0| 977348578|    146|
|    1340|   4.0|1010690174|    146|
|    2143|   2.0| 977348665|    146|
|    2070|   2.0| 979939838|    146|
|    2144|   3.0| 979940007|    146|
+--------+------+----------+-------+
only showing top 20 rows



24/04/12 05:11:35 WARN TaskSetManager: Stage 5 contains a task of very large size (11175 KiB). The maximum recommended task size is 1000 KiB.


1000209


In [ ]:
#preparation des donn'ees ratings
# Nous verrez que le champ timestamp est un timestamp UNIX en secondes. Elasticsearch prend les timestamps en millisecondes, 
# vous utiliserez donc quelques opérations DataFrame pour convertir les timestamps en millisecondes.

In [8]:
from pyspark.sql.functions import col
# Sélectionner les colonnes nécessaires et effectuer le casting sur la colonne "timestamp"
ratings = es_ratings_df.select(
    col("user_id"),
    col("movie_id"),
    col("rating"),
    (col("timestamp").cast("long") * 1000).alias("timestamp")
)

# Afficher les premières lignes du DataFrame modifié
ratings.show(5)

24/04/12 05:20:34 WARN TaskSetManager: Stage 8 contains a task of very large size (11175 KiB). The maximum recommended task size is 1000 KiB.


+-------+--------+------+-------------+
|user_id|movie_id|rating|    timestamp|
+-------+--------+------+-------------+
|    146|    2066|   3.0|1042049967000|
|    146|     804|   1.0| 977348138000|
|    146|    1193|   4.0| 979940868000|
|    146|    1267|   4.0| 977349660000|
|    146|    1269|   5.0| 977434285000|
+-------+--------+------+-------------+
only showing top 5 rows



In [9]:
# Requête Elasticsearch pour récupérer tous les documents de l'index "movies"
query = {
    "query": {
        "match_all": {}
    },
    "size": 3883
}
# Récupération des données Elasticsearch
es_response = client.search(index="movies", body=query)  # Msize fait reference au nombre de documents de l' index movies

# Convertir les résultats en une liste de dictionnaires
documents = [hit["_source"] for hit in es_response["hits"]["hits"]]

# Création d'une session Spark
spark = SparkSession.builder \
    .appName("Chargement des données Elasticsearch vers PySpark DataFrame") \
    .getOrCreate()

# Convertir la liste de dictionnaires en DataFrame Spark
es_movies_df = spark.createDataFrame(documents)

# Afficher les premières lignes du DataFrame
es_movies_df.show()

# Calculer la taille du DataFrame
print("Taille du DataFrame :", es_movies_df.count())

# N'oubliez pas de fermer la session Spark lorsque vous avez terminé
# spark.stop()


24/04/12 05:20:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+--------------------+--------+--------------------+
|              genres|movie_id|               title|
+--------------------+--------+--------------------+
|[Animation, Child...|       1|    Toy Story (1995)|
|[Adventure, Child...|       2|      Jumanji (1995)|
|   [Comedy, Romance]|       3|Grumpier Old Men ...|
|     [Comedy, Drama]|       4|Waiting to Exhale...|
|            [Comedy]|       5|Father of the Bri...|
|[Action, Crime, T...|       6|         Heat (1995)|
|   [Comedy, Romance]|       7|      Sabrina (1995)|
|[Adventure, Child...|       8| Tom and Huck (1995)|
|            [Action]|       9| Sudden Death (1995)|
|[Action, Adventur...|      10|    GoldenEye (1995)|
|[Comedy, Drama, R...|      11|American Presiden...|
|    [Comedy, Horror]|      12|Dracula: Dead and...|
|[Animation, Child...|      13|        Balto (1995)|
|             [Drama]|      14|        Nixon (1995)|
|[Action, Adventur...|      15|Cutthroat Island ...|
|   [Drama, Thriller]|      16|       Casino (

Taille du DataFrame : 3883


In [ ]:
# Vous remarquerez peut-être aussi que les titres des films contiennent l'année de sortie. 
# Il serait utile de disposer de ce champ dans votre index de recherche pour filtrer les résultats 
# (par exemple, si vous souhaitez filtrer nos recommandations pour n'inclure que les films les plus récents).

In [11]:
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, StructType, StructField

# Définir une UDF pour extraire l'année de sortie du titre des films
def extract_year_fn(title):
    result = re.search("\(\d{4}\)", title)
    try:
        if result:
            group = result.group()
            year = group[1:-1]
            start_pos = result.start()
            title = title[:start_pos-1]
            return (title, year)
        else:
            return (title, "1970")
    except:
        print(title)

# Enregistrer la fonction UDF
extract_year = udf(extract_year_fn, StructType([
    StructField("title", StringType(), True),
    StructField("release_date", StringType(), True)
]))

# Appliquer la fonction UDF à votre DataFrame des films
es_movies_with_year = es_movies_df.withColumn("title", extract_year("title").title)\
    .withColumn("release_date", extract_year("title").release_date)

# Afficher les données des films nettoyées
print("Données des films nettoyées :")
es_movies_with_year.show(5, truncate=False)


Données des films nettoyées :


+--------------------------------+--------+---------------------------+------------+
|genres                          |movie_id|title                      |release_date|
+--------------------------------+--------+---------------------------+------------+
|[Animation, Children's, Comedy] |1       |Toy Story                  |1970        |
|[Adventure, Children's, Fantasy]|2       |Jumanji                    |1970        |
|[Comedy, Romance]               |3       |Grumpier Old Men           |1970        |
|[Comedy, Drama]                 |4       |Waiting to Exhale          |1970        |
|[Comedy]                        |5       |Father of the Bride Part II|1970        |
+--------------------------------+--------+---------------------------+------------+
only showing top 5 rows



In [ ]:

# Créer des index Elasticsearch, avec des mappings pour les utilisateurs, les films et les événements de notation

# Dans Elasticsearch, un "index" est à peu près similaire à une "base de données" ou à une "table de base de données". 
# Le schéma d'un index s'appelle un mappage d'index.

# Bien qu'Elasticsearch prenne en charge le mappage dynamique, il est conseillé de spécifier explicitement le mappage 
# lors de la création d'un index si vous savez à quoi ressemblent vos données.

# Pour les besoins de votre moteur de recommandation, cela est également nécessaire pour que vous puissiez spécifier 
# le champ vectoriel qui contiendra le "modèle" de recommandation (c'est-à-dire les vecteurs de facteurs). 
# Lors de la création d'un champ vectoriel, vous devez fournir explicitement la dimension du vecteur, 
# de sorte qu'il ne peut s'agir d'un mappage dynamique.


In [ ]:

# Chargement des DataFrames Ratings et Movies dans Elasticsearch

# Tout d'abord, vous allez écrire les données d'évaluation dans Elasticsearch. 
# Notez que vous pouvez simplement utiliser le connecteur Spark Elasticsearch pour écrire un DataFrame avec 
# l'API native Spark datasource en spécifiant format("es")


In [12]:
# set the factor vector dimension for the recommendation model
VECTOR_DIM = 20

create_ratings = {
    "mappings": {
        "properties": {
            "timestamp": {
                "type": "date"
            },
            "userId": {
                "type": "integer"
            },
            "movieId": {
                "type": "integer"
            },
            "rating": {
                "type": "double"
            }
        }  
    }
}

create_users = {
    "mappings": {
        "properties": {
            "userId": {
                "type": "integer"
            },
            "model_factor": {
                "type": "dense_vector",
                "dims" : VECTOR_DIM
            },
            "model_version": {
                "type": "keyword"
            },
            "model_timestamp": {
                "type": "date"
            }
        }
    }
}

create_movies = {
    "mappings": {
        "properties": {
            "movieId": {
                "type": "integer"
            },
            "tmdbId": {
                "type": "keyword"
            },
            "genres": {
                "type": "keyword"
            },
            "release_date": {
                "type": "date",
                "format": "year"
            },
            "model_factor": {
                "type": "dense_vector",
                "dims" : VECTOR_DIM
            },
            "model_version": {
                "type": "keyword"
            },
            "model_timestamp": {
                "type": "date"
            }          
        }
    }
}

# create indices with the settings and mappings above
res_ratings = client.indices.create(index="ratingsdf", body=create_ratings)
res_users = client.indices.create(index="usersdf", body=create_users)
res_movies = client.indices.create(index="moviesdf", body=create_movies)

print("Created indices:")
print(res_ratings)
print(res_users)
print(res_movies)


Created indices:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'ratingsdf'}
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'usersdf'}
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'moviesdf'}


In [18]:
from elasticsearch import helpers

# Convertir les données de notation en un format compatible avec la méthode bulk
rating_documents =[
    {
        "_index": "ratingsdf",  # Index Elasticsearch
        "_source": {           # Source des données
            "userId": row.user_id,
            "movieId": row.movie_id,
            "rating": row.rating,
            "timestamp": row.timestamp
        }
    }
    for row in ratings.collect()  # Parcourir les lignes du DataFrame
]

# Utiliser la méthode bulk pour insérer les documents dans Elasticsearch
success, _ = helpers.bulk(client, rating_documents, index="ratingsdf")

# Vérifier si l'opération s'est bien déroulée
if success:
    print("Les données de notation ont été insérées avec succès dans Elasticsearch.")
else:
    print("Une erreur s'est produite lors de l'insertion des données de notation dans Elasticsearch.")


24/04/12 06:08:33 WARN TaskSetManager: Stage 18 contains a task of very large size (11175 KiB). The maximum recommended task size is 1000 KiB.


Les données de notation ont été insérées avec succès dans Elasticsearch.


In [ ]:
# write movie data, specifying the DataFrame column to use as the id mapping
movie_data.write.format("es").option("es.mapping.id", "movieId").save("movies")
num_movies_df = movie_data.count()
num_movies_es = es.count(index="movies")['count']
# check load went ok
print("Movie DF count: {}".format(num_movies_df))
print("ES index count: {}".format(num_movies_es))

In [ ]:
# ETAPE 4: Choix du type de modèle 

#     En fonction de notre cas d'utilisation, nous devons adopte le type de modèle de recommandation le plus approprié. 
# Pour commencer, Nous pouvez opter pour :
#         Filtrage collaboratif basé sur les utilisateurs (User-Based Collaborative Filtering) : 
# recommande des éléments à un utilisateur basé sur les préférences des utilisateurs similaires.
#         Filtrage collaboratif basé sur les articles (Item-Based Collaborative Filtering) : 
# recommande des éléments similaires à ceux que l'utilisateur a aimés dans le passé.
#         Décomposition en valeurs singulières (Singular Value Decomposition - SVD) : 
# décompose la matrice des évaluations utilisateur-item en matrices de caractéristiques pour capturer les relations latentes.
#         Factorisation de matrices non négatives (Non-Negative Matrix Factorization - NMF) : 
# similaire à SVD mais contraint les matrices de caractéristiques à être non négatives.

In [ ]:
# Comme piste intéressante nous pouvons trouver des groupes d'utilisateurs ayant le même ressenti (films, genres, notes, tags).
# Un utilisateur d'un de ces groupes sera plus disposé à apprécier les films plébiscités  par les autres membres du groupe. 
# il s' agit donc du Filtrage collaboratif basé sur les utilisateurs, nous implementerons donc ce type de filtrage.

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit  # Importer lit depuis pyspark.sql.functions
from pyspark.ml.recommendation import ALS

# Créer une session Spark
spark = SparkSession.builder.appName("Movie_Recommendation").getOrCreate()

# Construction du modèle ALS
als = ALS(userCol="user_id", itemCol="movie_id", ratingCol="rating",
          coldStartStrategy="drop", nonnegative=True)

# Entraînement du modèle
model = als.fit(es_ratings_df)

# Obtenir les recommandations pour tous les utilisateurs
user_recs = model.recommendForAllUsers(5)  # Recommender 5 films pour chaque utilisateur

# Convertir les recommandations en DataFrame et les afficher
user_recs_dfs = []
for row in user_recs.collect():
    user_id = row.user_id
    recommendations = row.recommendations
    recommendations_df = spark.createDataFrame(recommendations)
    recommendations_df = recommendations_df.withColumn("user_id", lit(user_id))
    user_recs_dfs.append(recommendations_df)

# Afficher les recommandations pour chaque utilisateur
for df in user_recs_dfs:
    df.show()


24/04/12 00:25:39 WARN TaskSetManager: Stage 409 contains a task of very large size (11175 KiB). The maximum recommended task size is 1000 KiB.
24/04/12 00:25:43 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 409 (TID 931): Attempting to kill Python Worker
24/04/12 00:25:43 WARN TaskSetManager: Stage 410 contains a task of very large size (11175 KiB). The maximum recommended task size is 1000 KiB.


+--------+-----------------+-------+
|movie_id|           rating|user_id|
+--------+-----------------+-------+
|     572|5.620935440063477|      1|
|     557|4.816728591918945|      1|
|    3233|4.760732650756836|      1|
|     527|4.582545280456543|      1|
|     989|4.544499397277832|      1|
+--------+-----------------+-------+

+--------+------------------+-------+
|movie_id|            rating|user_id|
+--------+------------------+-------+
|     572|  5.24021577835083|      3|
|    1851|4.6411638259887695|      3|
|     318| 4.557138442993164|      3|
|     811| 4.532729625701904|      3|
|     110|  4.52290678024292|      3|
+--------+------------------+-------+

+--------+-----------------+-------+
|movie_id|           rating|user_id|
+--------+-----------------+-------+
|    2309|4.416167736053467|      5|
|     557|4.412998199462891|      5|
|    1423|4.266970157623291|      5|
|    1138|4.149060249328613|      5|
|    1149|4.116970539093018|      5|
+--------+-----------------

+--------+------------------+-------+
|movie_id|            rating|user_id|
+--------+------------------+-------+
|     572| 5.221408367156982|    758|
|    1851|   4.7882399559021|    758|
|     318| 4.619048118591309|    758|
|    2905| 4.506421089172363|    758|
|    3233|4.5061163902282715|    758|
+--------+------------------+-------+

+--------+------------------+-------+
|movie_id|            rating|user_id|
+--------+------------------+-------+
|     557| 5.320233345031738|    760|
|     572|5.2578606605529785|    760|
|    2309| 5.200313091278076|    760|
|      53| 5.035312652587891|    760|
|     989| 4.967796325683594|    760|
+--------+------------------+-------+

+--------+------------------+-------+
|movie_id|            rating|user_id|
+--------+------------------+-------+
|    3382| 5.584162712097168|    761|
|     572| 5.388779640197754|    761|
|     598| 5.360081672668457|    761|
|    1851|5.3463239669799805|    761|
|     989| 5.296114921569824|    761|
+--------+

+--------+-----------------+-------+
|movie_id|           rating|user_id|
+--------+-----------------+-------+
|     572|4.871671199798584|   1756|
|    2197|4.412827014923096|   1756|
|    3612|4.327188014984131|   1756|
|     985|  4.3156418800354|   1756|
|    1035|4.314403533935547|   1756|
+--------+-----------------+-------+

+--------+-----------------+-------+
|movie_id|           rating|user_id|
+--------+-----------------+-------+
|     572|5.181822299957275|   1759|
|     598|4.874283313751221|   1759|
|     989| 4.83081579208374|   1759|
|    3382|4.813741683959961|   1759|
|    2127| 4.78848934173584|   1759|
+--------+-----------------+-------+

+--------+------------------+-------+
|movie_id|            rating|user_id|
+--------+------------------+-------+
|     572| 4.516287326812744|   1766|
|    1741| 4.186757564544678|   1766|
|    2562| 4.128747940063477|   1766|
|    1851|3.9887070655822754|   1766|
|    3092|3.9560165405273438|   1766|
+--------+------------------

+--------+------------------+-------+
|movie_id|            rating|user_id|
+--------+------------------+-------+
|     598| 4.482316017150879|   2245|
|    3382| 4.401245594024658|   2245|
|     108| 4.368863582611084|   2245|
|     557|4.2837748527526855|   2245|
|    2964| 4.243851184844971|   2245|
+--------+------------------+-------+

+--------+-----------------+-------+
|movie_id|           rating|user_id|
+--------+-----------------+-------+
|     572|4.658445835113525|   2246|
|    1851|4.448624134063721|   2246|
|    3245|4.405317306518555|   2246|
|    3233| 4.31337308883667|   2246|
|    2905|4.310996055603027|   2246|
+--------+-----------------+-------+

+--------+-----------------+-------+
|movie_id|           rating|user_id|
+--------+-----------------+-------+
|     572|4.993715286254883|   2247|
|     557|4.776267051696777|   2247|
|    2503|4.411121845245361|   2247|
|    1207| 4.39661979675293|   2247|
|     912|4.355991840362549|   2247|
+--------+-----------------

...

+--------+-----------------+-------+
|movie_id|           rating|user_id|
+--------+-----------------+-------+
|     572|5.062793254852295|   5840|
|    1851|4.421513557434082|   5840|
|    3233|4.400609016418457|   5840|
|     557| 4.34506368637085|   5840|
|     527|4.311950206756592|   5840|
+--------+-----------------+-------+

+--------+------------------+-------+
|movie_id|            rating|user_id|
+--------+------------------+-------+
|     572|5.1593170166015625|   5842|
|     853|4.6917924880981445|   5842|
|    1198| 4.604874610900879|   5842|
|     260| 4.532989978790283|   5842|
|     318| 4.517976760864258|   5842|
+--------+------------------+-------+



+--------+------------------+-------+
|movie_id|            rating|user_id|
+--------+------------------+-------+
|     557|5.1516008377075195|   5846|
|    2503| 4.871824741363525|   5846|
|      53| 4.864509105682373|   5846|
|     572| 4.818161487579346|   5846|
|    3245| 4.775269508361816|   5846|
+--------+------------------+-------+

+--------+------------------+-------+
|movie_id|            rating|user_id|
+--------+------------------+-------+
|     572| 4.796111106872559|   5848|
|     557| 4.574593544006348|   5848|
|    3245| 4.445799350738525|   5848|
|      53|4.4409708976745605|   5848|
|    3233| 4.411890983581543|   5848|
+--------+------------------+-------+

+--------+------------------+-------+
|movie_id|            rating|user_id|
+--------+------------------+-------+
|    3245| 4.983151912689209|   5849|
|     572| 4.979135990142822|   5849|
|     811| 4.789872169494629|   5849|
|    1851| 4.732543468475342|   5849|
|    1741|4.7191081047058105|   5849|
+--------+

+--------+------------------+-------+
|movie_id|            rating|user_id|
+--------+------------------+-------+
|     572| 2.905411958694458|   5850|
|    2129|2.6077444553375244|   5850|
|    3233| 2.408022880554199|   5850|
|     557| 2.304615020751953|   5850|
|    3003| 2.269157648086548|   5850|
+--------+------------------+-------+

+--------+-----------------+-------+
|movie_id|           rating|user_id|
+--------+-----------------+-------+
|    2309| 4.84959602355957|   5853|
|     557| 4.59362268447876|   5853|
|    3338|4.491434097290039|   5853|
|    1002|4.470223903656006|   5853|
|    1787|4.464378356933594|   5853|
+--------+-----------------+-------+

+--------+-----------------+-------+
|movie_id|           rating|user_id|
+--------+-----------------+-------+
|     572|5.081118106842041|   5855|
|    1851|5.030388832092285|   5855|
|    3245|4.760630130767822|   5855|
|    3233|4.725485324859619|   5855|
|    2197|4.703383922576904|   5855|
+--------+-----------------

+--------+-----------------+-------+
|movie_id|           rating|user_id|
+--------+-----------------+-------+
|    1420|4.993624687194824|   5877|
|    2309|4.945960998535156|   5877|
|     887|4.896255016326904|   5877|
|     853|4.839127540588379|   5877|
|     572|4.801061153411865|   5877|
+--------+-----------------+-------+

+--------+------------------+-------+
|movie_id|            rating|user_id|
+--------+------------------+-------+
|     572| 4.972508430480957|   5878|
|    1851| 4.720009803771973|   5878|
|    3092|4.6449103355407715|   5878|
|    3245| 4.625307559967041|   5878|
|     557| 4.616247177124023|   5878|
+--------+------------------+-------+



+--------+------------------+-------+
|movie_id|            rating|user_id|
+--------+------------------+-------+
|     572| 5.202920913696289|   5881|
|     557|4.8445940017700195|   5881|
|    1851|4.7367353439331055|   5881|
|    2562| 4.722244739532471|   5881|
|     989| 4.690458297729492|   5881|
+--------+------------------+-------+

+--------+------------------+-------+
|movie_id|            rating|user_id|
+--------+------------------+-------+
|     853| 4.531094074249268|   5882|
|    3382| 4.478959560394287|   5882|
|     260| 4.475204944610596|   5882|
|     572|4.4642486572265625|   5882|
|     811| 4.423866271972656|   5882|
+--------+------------------+-------+

+--------+-----------------+-------+
|movie_id|           rating|user_id|
+--------+-----------------+-------+
|    2309|5.199183940887451|   5883|
|     296|4.655754566192627|   5883|
|     572|4.606681823730469|   5883|
|    2905| 4.52636194229126|   5883|
|     787| 4.52234411239624|   5883|
+--------+--------